# **Mantenimiento de Datos OCDE en el SGBD**

* El Siguiente programa extraerá los datos de la OCDE por medio de una API y generará una consulta en el SGBD de tres tablas alojados al SGBD: investigadores_mujeres_OCDE, investigadores_1000 y investigadores_SCIMago, los resultados de ambos procedimientos se compararan los datos y si existen diferencia se generará un registro de sus respectivas diferencias.

* En el caso de existir diferencias el programa modificará el dato al mas actualizado.

In [1]:
# Importamos las librerias necesarias
import requests
import openpyxl # para exportar los resultados a un excel
import csv
import os
import pandas as pd

## Obtenemos los indicadores de ciencia y tecnología OCDE cada mil personas trabajando desde el 2012
https://data-explorer.oecd.org/vis?tm=main%20science&pg=0&snb=1&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_MSTI%40DF_MSTI&df[ag]=OECD.STI.STP&df[vs]=1.1&pd=2012%2C&dq=.A.T_RS.10P3EMP..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

In [2]:
nombre_del_archivo = "OCDE_KTrabajadores.csv"                             # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos a conectarnos con la API para obtener el descargable.")

    # Define the URL of the request
    url = "https://sdmx.oecd.org/public/rest/data/OECD.STI.STP,DSD_MSTI@DF_MSTI,1.1/.A.T_RS.10P3EMP..?startPeriod=2012&dimensionAtObservation=AllDimensions"

    # Establece las cabeceras para solicitar datos formato csv
    parametros = {
        'Accept': 'application/vnd.sdmx.data+csv; charset=utf-8',
    }

    # Realizar la solicitud GET
    response = requests.get(url, headers=parametros)

    # Verifique el código de estado de la respuesta
    if response.status_code == 200:
        # Si la respuesta está en formato CSV, puede guardarla en un archivo o procesarla según sea necesario
        with open(nombre_del_archivo, 'w') as file:
            file.write(response.text)
        print("Descarga Exitosa")
    else:
        print(f"Error de solicitud: código de estado {response.status_code}")

def extraccion_ocde(ruta_archivo_csv):
    with open(ruta_archivo_csv, newline='', encoding='utf-8') as archivo_csv:
        lector = csv.reader(archivo_csv)
        datos = list(lector)

    datos_por_pais = {}
    for fila in datos[1:]:  # Ignora el encabezado
        if len(fila) > 8:  # Asegura que la fila tenga suficientes columnas
            pais = fila[1]  # La abreviatura del país está en la segunda columna
            ano = fila[7]  # El año está en la octava columna
            valor = fila[8]  # El dato está en la novena columna
            if pais not in datos_por_pais:
                datos_por_pais[pais] = {}
            try:
                datos_por_pais[pais][ano] = float(valor)
            except ValueError:
                continue  # Ignora los valores que no se pueden convertir a flotante

    return datos_por_pais

# Lista de códigos ISO 3166-1 alfa-3 de países y sus nombres completos para mostrar
paises_ocde = [
    ("AUT", "Austria"), ("BEL", "Bélgica"), ("CAN", "Canadá"), ("CHL", "Chile"), ("CRI", "Costa Rica"), 
    ("CZE", "República Checa"), ("DNK", "Dinamarca"), ("EST", "Estonia"), ("FIN", "Finlandia"),
    ("FRA", "Francia"), ("DEU", "Alemania"), ("GRC", "Grecia"), ("HUN", "Hungría"), ("ISL", "Islandia"),
    ("IRL", "Irlanda"), ("ITA", "Italia"), ("JPN", "Japón"), ("KOR", "Corea del Sur"),
    ("LVA", "Letonia"), ("LTU", "Lituania"), ("LUX", "Luxemburgo"), ("MEX", "México"), ("NLD", "Países Bajos"),
    ("NZL", "Nueva Zelanda"), ("NOR", "Noruega"), ("POL", "Polonia"), ("PRT", "Portugal"), ("SVK", "Eslovaquia"),
    ("SVN", "Eslovenia"), ("ESP", "España"), ("SWE", "Suecia"), ("CHE", "Suiza"), ("TUR", "Turquía"),
    ("GBR", "Reino Unido"), ("USA", "Estados Unidos"), ("EU27_2020", "Unión Europea (27)") ,("OECD", "OECD - Total")
]

# Llamada a la función de extracción con la ruta del archivo CSV
datos_por_pais = extraccion_ocde(nombre_del_archivo)

# Convertir los datos extraídos a una lista de diccionarios para crear un DataFrame
lista_datos = []
for pais, datos_anuales in datos_por_pais.items():
    for ano, valor in datos_anuales.items():
        lista_datos.append({'Año': int(ano), 'País': pais, 'Valor': valor})

# Crear el DataFrame a partir de la lista de diccionarios
df_ocde_KTrabajadores = pd.DataFrame(lista_datos)

# Convertir la lista de tuplas de países en un diccionario
codigo_a_nombre = dict(paises_ocde)

# Mapear los códigos de país a nombres completos en el DataFrame
df_ocde_KTrabajadores['País'] = df_ocde_KTrabajadores['País'].map(codigo_a_nombre)

# Eliminar los registros donde 'País' es NaN
df_ocde_KTrabajadores.dropna(subset=['País'], inplace=True)

# Eliminar directamente los registros del año 2022
df_ocde_KTrabajadores = df_ocde_KTrabajadores[df_ocde_KTrabajadores['Año'] != 2022]

# Mostrar el DataFrame resultante
print(df_ocde_KTrabajadores)
print(df_ocde_KTrabajadores.dtypes)



El archivo 'OCDE_KTrabajadores.csv' está presente en el directorio actual.
      Año        País     Valor
0    2012      Canadá  9.131744
1    2013      Canadá  9.112551
2    2014      Canadá  9.003252
3    2015      Canadá  8.990800
4    2016      Canadá  8.688412
..    ...         ...       ...
392  2017  Eslovaquia  6.418203
393  2018  Eslovaquia  6.751194
394  2019  Eslovaquia  6.942987
395  2020  Eslovaquia  7.201290
396  2021  Eslovaquia  7.345063

[339 rows x 3 columns]
Año        int64
País      object
Valor    float64
dtype: object


## Obtenemos los indicadores de ciencia y tecnología OCDE con respecto al PIB desde el 2017
https://data-explorer.oecd.org/vis?pg=0&bp=true&snb=6&tm=science&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_MSTI%40DF_MSTI&df[ag]=OECD.STI.STP&df[vs]=1.0&pd=2017%2C&dq=.A.G.PT_B1GQ..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

In [3]:
nombre_del_archivo = "OCDE_PIB.csv"                             # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos a conectarnos con la API para obtener el descargable.")

    # Define the URL of the request
    url = "https://sdmx.oecd.org/public/rest/data/OECD.STI.STP,DSD_MSTI@DF_MSTI,1.0/.A.G.PT_B1GQ..?startPeriod=2017&dimensionAtObservation=AllDimensions"

    # Establece las cabeceras para solicitar datos formato csv
    parametros = {
        'Accept': 'application/vnd.sdmx.data+csv; charset=utf-8',
    }

    # Realizar la solicitud GET
    response = requests.get(url, headers=parametros)

    # Verifique el código de estado de la respuesta
    if response.status_code == 200:
        # Si la respuesta está en formato CSV, puede guardarla en un archivo o procesarla según sea necesario
        with open(nombre_del_archivo, 'w') as file:
            file.write(response.text)
        print("Descarga Exitosa")
    else:
        print(f"Error de solicitud: código de estado {response.status_code}")

def extraccion_ocde(ruta_archivo_csv):
    with open(ruta_archivo_csv, newline='', encoding='utf-8') as archivo_csv:
        lector = csv.reader(archivo_csv)
        datos = list(lector)

    datos_por_pais = {}
    for fila in datos[1:]:  # Ignora el encabezado
        if len(fila) > 8:  # Asegura que la fila tenga suficientes columnas
            pais = fila[1]  # La abreviatura del país está en la segunda columna
            ano = fila[7]  # El año está en la octava columna
            valor = fila[8]  # El dato está en la novena columna
            if pais not in datos_por_pais:
                datos_por_pais[pais] = {}
            try:
                datos_por_pais[pais][ano] = float(valor)
            except ValueError:
                continue  # Ignora los valores que no se pueden convertir a flotante

    return datos_por_pais

# Lista de códigos ISO 3166-1 alfa-3 de países y sus nombres completos para mostrar
paises_ocde = [
    ("AUT", "Austria"), ("BEL", "Bélgica"), ("CAN", "Canadá"), ("CHL", "Chile"), ("CRI", "Costa Rica"), 
    ("CZE", "República Checa"), ("DNK", "Dinamarca"), ("EST", "Estonia"), ("FIN", "Finlandia"),
    ("FRA", "Francia"), ("DEU", "Alemania"), ("GRC", "Grecia"), ("HUN", "Hungría"), ("ISL", "Islandia"),
    ("IRL", "Irlanda"), ("ITA", "Italia"), ("JPN", "Japón"), ("KOR", "Corea del Sur"),
    ("LVA", "Letonia"), ("LTU", "Lituania"), ("LUX", "Luxemburgo"), ("MEX", "México"), ("NLD", "Países Bajos"),
    ("NZL", "Nueva Zelanda"), ("NOR", "Noruega"), ("POL", "Polonia"), ("PRT", "Portugal"), ("SVK", "Eslovaquia"),
    ("SVN", "Eslovenia"), ("ESP", "España"), ("SWE", "Suecia"), ("CHE", "Suiza"), ("TUR", "Turquía"),
    ("GBR", "Reino Unido"), ("USA", "Estados Unidos"), ("EU27_2020", "Unión Europea (27)") ,("OECD", "OECD - Total")
]

# Llamada a la función de extracción con la ruta del archivo CSV
datos_por_pais = extraccion_ocde(nombre_del_archivo)

# Convertir los datos extraídos a una lista de diccionarios para crear un DataFrame
lista_datos = []
for pais, datos_anuales in datos_por_pais.items():
    for ano, valor in datos_anuales.items():
        lista_datos.append({'Año': int(ano), 'País': pais, 'Valor': valor})

# Crear el DataFrame a partir de la lista de diccionarios
df_ocde_pib = pd.DataFrame(lista_datos)

# Convertir la lista de tuplas de países en un diccionario
codigo_a_nombre = dict(paises_ocde)

# Mapear los códigos de país a nombres completos en el DataFrame
df_ocde_pib['País'] = df_ocde_pib['País'].map(codigo_a_nombre)

# Eliminar los registros donde 'País' es NaN
df_ocde_pib.dropna(subset=['País'], inplace=True)

# Eliminar directamente los registros del año 2022
df_ocde_pib = df_ocde_pib[df_ocde_pib['Año'] != 2022]

# Mostrar el DataFrame resultante
print(df_ocde_pib)

El archivo 'OCDE_PIB.csv' está presente en el directorio actual.
      Año             País     Valor
4    2017         Islandia  2.084135
5    2018         Islandia  2.000809
6    2019         Islandia  2.341648
7    2020         Islandia  2.490692
8    2021         Islandia  2.805417
..    ...              ...       ...
215  2021  República Checa  1.996006
216  2017       Costa Rica  0.430539
217  2018       Costa Rica  0.371300
218  2020       Costa Rica  0.330022
219  2021       Costa Rica  0.275308

[175 rows x 3 columns]


## Obtenemos los indicadores de ciencia y tecnología OCDE de investigadores e investigadoras desde el 2010
https://data-explorer.oecd.org/vis?tm=main%20science&pg=0&snb=1&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_MSTI%40DF_MSTI&df[ag]=OECD.STI.STP&df[vs]=1.1&pd=2010%2C&dq=.A.T_RS%2BT_WRS.PS..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

In [4]:
nombre_del_archivo = "OCDE_Inv.csv"                             # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos a conectarnos con la API para obtener el descargable.")

    # Define the URL of the request
    url = "https://sdmx.oecd.org/public/rest/data/OECD.STI.STP,DSD_MSTI@DF_MSTI,1.1/.A.T_RS+T_WRS.PS..?startPeriod=2010&dimensionAtObservation=AllDimensions"

    # Establece las cabeceras para solicitar datos formato csv
    parametros = {
        'Accept': 'application/vnd.sdmx.data+csv; charset=utf-8',
    }

    # Realizar la solicitud GET
    response = requests.get(url, headers=parametros)

    # Verifique el código de estado de la respuesta
    if response.status_code == 200:
        # Si la respuesta está en formato CSV, puede guardarla en un archivo o procesarla según sea necesario
        with open(nombre_del_archivo, 'w') as file:
            file.write(response.text)
        print("Descarga Exitosa")
    else:
        print(f"Error de solicitud: código de estado {response.status_code}")

def extraccion_ocde(ruta_archivo_csv):
    with open(ruta_archivo_csv, newline='', encoding='utf-8') as archivo_csv:
        lector = csv.reader(archivo_csv)
        datos = list(lector)

    datos_por_pais_y_categoria = {}
    for fila in datos[1:]:  # Ignora el encabezado
        if len(fila) > 8:  # Asegura que la fila tenga suficientes columnas
            pais = fila[1]  # La abreviatura del país está en la segunda columna
            categoria = fila[3]  # La categoría (T_WRS o T_RS) está en la cuarta columna
            ano = fila[7]  # El año está en la octava columna
            valor = fila[8]  # El dato está en la novena columna
            if pais not in datos_por_pais_y_categoria:
                datos_por_pais_y_categoria[pais] = {}
            if categoria not in datos_por_pais_y_categoria[pais]:
                datos_por_pais_y_categoria[pais][categoria] = {}
            try:
                datos_por_pais_y_categoria[pais][categoria][ano] = float(valor)
            except ValueError:
                continue  # Ignora los valores que no se pueden convertir a flotante

    return datos_por_pais_y_categoria

# Lista de códigos ISO 3166-1 alfa-3 de países y sus nombres completos para mostrar
paises_ocde = [
    ("AUT", "Austria"), ("BEL", "Bélgica"), ("CAN", "Canadá"), ("CHL", "Chile"), ("CRI", "Costa Rica"), 
    ("CZE", "República Checa"), ("DNK", "Dinamarca"), ("EST", "Estonia"), ("FIN", "Finlandia"),
    ("FRA", "Francia"), ("DEU", "Alemania"), ("GRC", "Grecia"), ("HUN", "Hungría"), ("ISL", "Islandia"),
    ("IRL", "Irlanda"), ("ITA", "Italia"), ("JPN", "Japón"), ("KOR", "Corea del Sur"),
    ("LVA", "Letonia"), ("LTU", "Lituania"), ("LUX", "Luxemburgo"), ("MEX", "México"), ("NLD", "Países Bajos"),
    ("NZL", "Nueva Zelanda"), ("NOR", "Noruega"), ("POL", "Polonia"), ("PRT", "Portugal"), ("SVK", "Eslovaquia"),
    ("SVN", "Eslovenia"), ("ESP", "España"), ("SWE", "Suecia"), ("CHE", "Suiza"), ("TUR", "Turquía"),
    ("GBR", "Reino Unido"), ("USA", "Estados Unidos"), ("EU27_2020", "Unión Europea (27)") ,("OECD", "OECD - Total")
]

# Llamada a la función de extracción con la ruta del archivo CSV
datos_por_pais_y_categoria = extraccion_ocde(nombre_del_archivo)

# Mostrar resultados para pais, categoría y calcula la proporción de investigadoras
lista_datos = []

for codigo_pais, nombre_pais in paises_ocde:
    if codigo_pais in datos_por_pais_y_categoria:
        datos_pais = datos_por_pais_y_categoria[codigo_pais]
        for ano in range(2010, 2022):  # Asume que los datos están disponibles para estos años
            valor_investigadores = datos_pais.get('T_RS', {}).get(str(ano))
            valor_investigadoras = datos_pais.get('T_WRS', {}).get(str(ano))
            if valor_investigadores and valor_investigadoras:
                proporcion = (valor_investigadoras / valor_investigadores) * 100
                lista_datos.append({'Año': ano, 'País': nombre_pais, 'Proporción': proporcion})

# Crear el DataFrame a partir de la lista
df_ocde_inv = pd.DataFrame(lista_datos)

# Eliminar los registros donde 'País' es NaN y del año 2022 en adelante
df_ocde_inv.dropna(subset=['País'], inplace=True)
df_ocde_inv = df_ocde_inv[df_ocde_inv['Año'] < 2022]

print(df_ocde_inv)       

El archivo 'OCDE_Inv.csv' está presente en el directorio actual.
      Año         País  Proporción
0    2011      Austria   28.989925
1    2013      Austria   29.594950
2    2015      Austria   29.493536
3    2017      Austria   30.059296
4    2019      Austria   30.392041
..    ...          ...         ...
302  2013  Reino Unido   38.098367
303  2014  Reino Unido   37.411822
304  2015  Reino Unido   38.589967
305  2016  Reino Unido   38.666093
306  2017  Reino Unido   38.749098

[307 rows x 3 columns]


In [6]:
# importamos las librerías respectivas y leemos la credencial de acceso
import psycopg2
import configparser
import pandas as pd
import numpy as np

config = configparser.ConfigParser()
config.read('credencial_login.conf')
database = config['postgresql']['database']
user     = config['postgresql']['user']
password = config['postgresql']['password']
host     = config['postgresql']['host']
port     = config['postgresql']['port']

## 1. **[Cantidad de investigadores(as) cada mil personas trabajando en países de la OCDE.](https://www.observa.minciencia.gob.cl/indicadores/comparacion-internacional/investigadoresas-cada-mil-personas-trabajando)**
 * **Tabla  SGBD:** investigadores_1000_long 
 * **Dashboard Tablau:** 020202 

In [7]:
def obtener_dataframe_de_tabla(connection):
    query = f'SELECT * FROM "investigadores_1000_long"'
    dataframe = pd.read_sql_query(query, con=connection)
    return dataframe

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        database=database,
        user=user,
        password=password,
        host=host,
        port=port
    )
    connection.autocommit = True
    print('Conexión iniciada') 

    # Obtener el DataFrame para una tabla específica
    df_sgbd_inv = obtener_dataframe_de_tabla(connection)
    df_sgbd_inv['valor'] = df_sgbd_inv['valor'].str.replace(',', '.')
    df_sgbd_inv['valor'] = pd.to_numeric(df_sgbd_inv['valor'], errors='coerce')
    # Redondear a 9 decimales
    df_sgbd_inv['valor'] = df_sgbd_inv['valor'].round(9)
    print(df_sgbd_inv)
    print(df_sgbd_inv.dtypes)

except psycopg2.DatabaseError as e:
    print(f"Error al conectarse a la base de datos: {e}")
finally:
    if connection:
        connection.close()
        print('Conexión finalizada')


Conexión iniciada
       año        paises      valor
0     2020       Austria  11.595234
1     2012       Austria   9.440949
2     2013       Austria   9.580048
3     2014       Austria  10.006573
4     2015       Austria  10.164880
...    ...           ...        ...
1291  2015  OECD...Total   8.326129
1292  2016  OECD...Total   8.331056
1293  2017  OECD...Total   8.580055
1294  2018  OECD...Total   8.891826
1295  2019  OECD...Total   9.070016

[1296 rows x 3 columns]
año         int64
paises     object
valor     float64
dtype: object
Conexión finalizada


c:\Users\jsepulveda\AppData\Local\miniconda3\envs\minciencia\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## 3. **[Gasto en I+D respecto al PIB en países de la OCDE.](https://www.observa.minciencia.gob.cl/indicadores/comparacion-internacional/gasto-en-id-respecto-al-pib)**
 * **Tabla BD:** Investigadores_SCIMago
 * modificar tabla: GastoID_Porcentaje_PIB
 * **Dashboard Tablau:** 020201

In [23]:
def obtener_dataframe_de_tabla(connection):
    query = f'SELECT * FROM "Investigadores_SCIMago"'
    dataframe = pd.read_sql_query(query, con=connection)
    return dataframe

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        database=database,
        user=user,
        password=password,
        host=host,
        port=port
    )
    connection.autocommit = True
    print('Conexión iniciada') 

    # Obtener el DataFrame para una tabla específica
    df_sgbd_pib = obtener_dataframe_de_tabla(connection)
    df_sgbd_pib['Año último dato OCDE'] = df_sgbd_pib['Año último dato OCDE'].astype('int64')
    print(df_sgbd_pib.head()) # muestra los 5 primeros resultados
    print(df_sgbd_pib.dtypes)

except psycopg2.DatabaseError as e:
    print(f"Error al conectarse a la base de datos: {e}")
finally:
    if connection:
        connection.close()
        print('Conexión finalizada')



Conexión iniciada
   Año último dato OCDE  Investigadores cada 1000 trab.  Citas promedio  \
0                  2020                        8.276472        1.897312   
1                  2020                       14.938975        1.749094   
2                  2020                        8.055574        1.620981   
3                  2020                       15.932689        1.624274   
4                  2020                       11.409391        1.335351   

              País  
0  República Checa  
1        Dinamarca  
2          Estonia  
3        Finlandia  
4          Francia  
Año último dato OCDE                int64
Investigadores cada 1000 trab.    float64
Citas promedio                    float64
País                               object
dtype: object
Conexión finalizada


c:\Users\jsepulveda\AppData\Local\miniconda3\envs\minciencia\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## 22. **[Porcentaje de investigadoras mujeres en países de la OCDE.](https://www.observa.minciencia.gob.cl/genero/comparacion-internacional/porcentaje-de-investigadoras-mujeres-en-paises-de-la-ocde)**
 * **Tabla BD:** Investigadores_Mujeres_OCDE
 * **Dashboard Tablau:** 50704

In [15]:
def obtener_dataframe_de_tabla(connection):
    query = f'SELECT * FROM "Investigadores_Mujeres_OCDE"'
    dataframe = pd.read_sql_query(query, con=connection)
    return dataframe

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        database=database,
        user=user,
        password=password,
        host=host,
        port=port
    )
    connection.autocommit = True
    print('Conexión iniciada') 

    # Obtener el DataFrame para una tabla específica
    df_sgbd_mujeres = obtener_dataframe_de_tabla(connection)
    df_sgbd_mujeres['Año'] = pd.to_numeric(df_sgbd_mujeres['Año'], errors='coerce')
    df_sgbd_mujeres['Porcentaje de mujeres investigadoras respecto al total de inves'] = pd.to_numeric(df_sgbd_mujeres['Porcentaje de mujeres investigadoras respecto al total de inves'], errors='coerce')
    # Redondear a 9 decimales
    df_sgbd_mujeres['Porcentaje de mujeres investigadoras respecto al total de inves'] = df_sgbd_mujeres['Porcentaje de mujeres investigadoras respecto al total de inves'].round(9)
    print(df_sgbd_mujeres)
    print(df_sgbd_mujeres.dtypes)

except psycopg2.DatabaseError as e:
    print(f"Error al conectarse a la base de datos: {e}")
finally:
    if connection:
        connection.close()
        print('Conexión finalizada')

Conexión iniciada
        País Inglés     País Español   Año  \
0           Austria          Austria  2013   
1           Belgium          Bélgica  2013   
2             Chile            Chile  2013   
3    Czech Republic  República Checa  2013   
4           Denmark        Dinamarca  2013   
..              ...              ...   ...   
212  United Kingdom      Reino Unido  2019   
213           Korea    Corea del Sur  2013   
214           Korea    Corea del Sur  2017   
215           Korea    Corea del Sur  2018   
216           Korea    Corea del Sur  2019   

     Porcentaje de mujeres investigadoras respecto al total de inves  
0                                            29.594950                
1                                            33.399934                
2                                            34.325995                
3                                            28.252468                
4                                            35.246043                
.. 

c:\Users\jsepulveda\AppData\Local\miniconda3\envs\minciencia\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# **Comparación**

## 1. **[Cantidad de investigadores(as) cada mil personas trabajando en países de la OCDE.](https://www.observa.minciencia.gob.cl/indicadores/comparacion-internacional/investigadoresas-cada-mil-personas-trabajando)**

In [9]:
# Renombrar las columnas de df_sgbd para que coincidan con las de df_ocde
df_sgbd_inv_renombrado = df_sgbd_inv.rename(columns={'año': 'Año', 'paises': 'País', 'valor': 'Valor'})

# Unir los DataFrames para compararlos
df_comparacion_inv = pd.merge(df_ocde_KTrabajadores, df_sgbd_inv_renombrado, on=['Año', 'País'], how='outer', suffixes=('_ocde', '_sgbd'))

# Asegurarse de que los valores sean numéricos
df_comparacion_inv['Valor_ocde'] = pd.to_numeric(df_comparacion_inv['Valor_ocde'], errors='coerce')
df_comparacion_inv['Valor_sgbd'] = pd.to_numeric(df_comparacion_inv['Valor_sgbd'], errors='coerce')

# Calcular la diferencia absoluta y el porcentaje de diferencia
df_comparacion_inv['Diferencia Abs'] = abs(df_comparacion_inv['Valor_ocde'] - df_comparacion_inv['Valor_sgbd'])
df_comparacion_inv['Porcentaje Dif'] = (df_comparacion_inv['Diferencia Abs'] / df_comparacion_inv['Valor_ocde']) * 100

# Reportar diferencias, valores faltantes y la diferencia porcentual
for _, fila in df_comparacion_inv.iterrows():
    año, país = fila['Año'], fila['País']
    valor_ocde, valor_sgbd = fila['Valor_ocde'], fila['Valor_sgbd']
    porcentaje_dif = fila['Porcentaje Dif']

    if pd.isna(valor_sgbd) and not pd.isna(valor_ocde):
        print(f"Valor no ingresado al SGBD para {país} año {año}, valor en OCDE: {valor_ocde}")
    elif pd.isna(valor_ocde) and not pd.isna(valor_sgbd):
        print(f"Falta valor en OCDE para {país} año {año}, valor en SGBD: {valor_sgbd}")
    elif not pd.isna(valor_ocde) and not pd.isna(valor_sgbd):
        print(f"Diferencia encontrada en {país} año {año}, OCDE: {valor_ocde}, SGBD: {valor_sgbd}, Diferencia porcentual: {porcentaje_dif:.2f}%")


Diferencia encontrada en Canadá año 2012, OCDE: 9.131744056, SGBD: 9.099101661, Diferencia porcentual: 0.36%
Diferencia encontrada en Canadá año 2012, OCDE: 9.131744056, SGBD: 9.099101661, Diferencia porcentual: 0.36%
Diferencia encontrada en Canadá año 2012, OCDE: 9.131744056, SGBD: 9.099101661, Diferencia porcentual: 0.36%
Diferencia encontrada en Canadá año 2012, OCDE: 9.131744056, SGBD: 9.099101661, Diferencia porcentual: 0.36%
Diferencia encontrada en Canadá año 2013, OCDE: 9.112551006, SGBD: 9.068540617, Diferencia porcentual: 0.48%
Diferencia encontrada en Canadá año 2013, OCDE: 9.112551006, SGBD: 9.068540617, Diferencia porcentual: 0.48%
Diferencia encontrada en Canadá año 2013, OCDE: 9.112551006, SGBD: 9.068540617, Diferencia porcentual: 0.48%
Diferencia encontrada en Canadá año 2013, OCDE: 9.112551006, SGBD: 9.068540617, Diferencia porcentual: 0.48%
Diferencia encontrada en Canadá año 2014, OCDE: 9.003251535, SGBD: 8.953048479, Diferencia porcentual: 0.56%
Diferencia encontra

## 3. **[Gasto en I+D respecto al PIB en países de la OCDE.](https://www.observa.minciencia.gob.cl/indicadores/comparacion-internacional/gasto-en-id-respecto-al-pib)**

In [24]:
# Renombrar las columnas de df_sgbd para que coincidan con las de df_ocde
df_sgbd_pib_renombrado = df_sgbd_pib.rename(columns={'Año último dato OCDE': 'Año', 'Investigadores cada 1000 trab.': 'Valor', 'País': 'País'})

# Unir los DataFrames para compararlos
df_comparacion_pib = pd.merge(df_ocde_pib, df_sgbd_pib_renombrado, on=['Año', 'País'], how='outer', suffixes=('_ocde', '_sgbd'))

# Asegurarse de que los valores sean numéricos
df_comparacion_pib['Valor_ocde'] = pd.to_numeric(df_comparacion_pib['Valor_ocde'], errors='coerce')
df_comparacion_pib['Valor_sgbd'] = pd.to_numeric(df_comparacion_pib['Valor_sgbd'], errors='coerce')

# Calcular la diferencia absoluta y el porcentaje de diferencia
df_comparacion_pib['Diferencia Abs'] = abs(df_comparacion_pib['Valor_ocde'] - df_comparacion_pib['Valor_sgbd'])
df_comparacion_pib['Porcentaje Dif'] = (df_comparacion_pib['Diferencia Abs'] / df_comparacion_pib['Valor_ocde']) * 100

# Reportar diferencias, valores faltantes y la diferencia porcentual
for _, fila in df_comparacion_pib.iterrows():
    año, país = fila['Año'], fila['País']
    valor_ocde, valor_sgbd = fila['Valor_ocde'], fila['Valor_sgbd']
    porcentaje_dif = fila['Porcentaje Dif']

    if pd.isna(valor_sgbd) and not pd.isna(valor_ocde):
        print(f"Valor no ingresado al SGBD para {país} año {año}, valor en OCDE: {valor_ocde}")
    elif pd.isna(valor_ocde) and not pd.isna(valor_sgbd):
        print(f"Falta valor en OCDE para {país} año {año}, valor en SGBD: {valor_sgbd}")
    elif not pd.isna(valor_ocde) and not pd.isna(valor_sgbd):
        print(f"Diferencia encontrada en {país} año {año}, OCDE: {valor_ocde}, SGBD: {valor_sgbd}, Diferencia porcentual: {porcentaje_dif:.2f}%")

Diferencia encontrada en Islandia año 2017, OCDE: 2.084134976, SGBD: 10.4219624, Diferencia porcentual: 400.06%
Valor no ingresado al SGBD para Islandia año 2018, valor en OCDE: 2.000808882
Valor no ingresado al SGBD para Islandia año 2019, valor en OCDE: 2.341648348
Diferencia encontrada en Islandia año 2020, OCDE: 2.49069205, SGBD: 10.4219623792578, Diferencia porcentual: 318.44%
Valor no ingresado al SGBD para Islandia año 2021, valor en OCDE: 2.805417494
Valor no ingresado al SGBD para Polonia año 2017, valor en OCDE: 1.037853655
Valor no ingresado al SGBD para Polonia año 2018, valor en OCDE: 1.206100129
Diferencia encontrada en Polonia año 2019, OCDE: 1.323352758, SGBD: 7.38117006, Diferencia porcentual: 457.76%
Diferencia encontrada en Polonia año 2020, OCDE: 1.386083638, SGBD: 7.58928135824755, Diferencia porcentual: 447.53%
Valor no ingresado al SGBD para Polonia año 2021, valor en OCDE: 1.431832956
Valor no ingresado al SGBD para Chile año 2017, valor en OCDE: 0.356788931
Val

## 22. **[Porcentaje de investigadoras mujeres en países de la OCDE.](https://www.observa.minciencia.gob.cl/genero/comparacion-internacional/porcentaje-de-investigadoras-mujeres-en-paises-de-la-ocde)**

# **Exportar resultados**

In [ ]:
# Especifica el nombre del archivo Excel donde deseas guardar los resultados
nombre_archivo_excel = 'Resultados_comparativos.xlsx'

# Guarda el DataFrame en un archivo Excel
df_comparacion_inv.to_excel(nombre_archivo_excel, sheet_name='OCDE', index=False)

print(f'Los datos han sido guardados con éxito en {nombre_archivo_excel}')

Los datos han sido guardados con éxito en Resultados_comparativos.xlsx
